In [ ]:
# !pip install -r ../requirements.txt > ../.log

In [19]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset
import librosa
import soundfile as sf
from moviepy import VideoFileClip
from openai import OpenAI
import assemblyai as aai
import yt_dlp

In [ ]:
def read_api_keys(file_path):
    api_keys = {}
    with open(file_path, 'r') as file:
        for line in file:
            key, value = line.strip().split('=')
            api_keys[key] = value.strip('"')
    return api_keys

# Path to the .api-key file
api_key_file_path = '../data/.api_keys'

# Read and parse the .api-key file
api_keys = read_api_keys(api_key_file_path)

# Access the API keys
openai_api_key = api_keys.get('openai')
assemblyai_api_key = api_keys.get('assemblyai')

# print("OpenAI API Key:", openai_api_key)

In [4]:
def convert_mp4_to_mp3(mp4_file, mp3_file):
    video_clip = VideoFileClip(mp4_file)
    audio_clip = video_clip.audio
    audio_clip.write_audiofile(mp3_file)
    audio_clip.close()
    video_clip.close()

mp4_file = '../data/video_easy.mp4'
audio_file = '../data/video_audio.mp3'
convert_mp4_to_mp3(mp4_file, audio_file)

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf61.1.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1920, 1094], 'bitrate': 309, 'fps': 60.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'Core Media Video', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc61.3.100 libx264'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 189, 'metadata': {'Metadata': '', 'handler_name': 'Core Media Audio', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 18.82, 'bitrate': 512, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [1920, 1094], 'video_bitrate': 309, 'video_fps':

MoviePy - Done.


In [5]:
async def download_audio(youtube_url: str, output_path: str):
    ydl_opts = {
        'format': 'bestvideo+bestaudio/best',
        'outtmpl': output_path,
        'noplaylist': True,         # Ensure only the single video is downloaded
        'overwrites': True,         # Overwrite the file if it already exists
        # 'cookies': cookies_path,    # Use cookies file
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])

youtube_url = 'https://www.youtube.com/watch?v=7vglAmBc9tY'     # detroit lion
audio_file = '../data/video.webm'
await download_audio(youtube_url, audio_file)

[youtube] Extracting URL: https://www.youtube.com/watch?v=7vglAmBc9tY
[youtube] 7vglAmBc9tY: Downloading webpage
[youtube] 7vglAmBc9tY: Downloading tv client config
[youtube] 7vglAmBc9tY: Downloading player 91201489
[youtube] 7vglAmBc9tY: Downloading tv player API JSON
[youtube] 7vglAmBc9tY: Downloading ios player API JSON
[youtube] 7vglAmBc9tY: Downloading m3u8 information
[info] 7vglAmBc9tY: Downloading 1 format(s): 396+251
Deleting existing file ../data/video.webm
[download] Destination: ../data/video.f396.mp4
[download] 100% of    7.53MiB in 00:00:00 at 23.02MiB/s    
[download] Destination: ../data/video.f251.webm
[download] 100% of    2.36MiB in 00:00:00 at 35.00MiB/s    
[Merger] Merging formats into "../data/video.webm"
Deleting original file ../data/video.f251.webm (pass -k to keep)
Deleting original file ../data/video.f396.mp4 (pass -k to keep)


In [11]:
def transcribe_audio_openai(audio_file_path):
    # Initialize the client
    client = OpenAI(api_key=openai_api_key)
    # Upload and transcribe the audio file
    with open(audio_file_path, 'rb') as audio_file:
        response = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file
        )
    # Extract and return the transcription
    return response.text

# Transcribe audio using OpenAI Whisper API
transcribed_text = transcribe_audio_openai(audio_file)
print("Transcribed Text:", transcribed_text)

Transcribed Text: ♪♪ Downtown Detroit, ready to open the 2024 season with higher hopes and more expectations than we've seen with this franchise in decades. Matter of fact, around the country, people are saying this is the team to beat. The Detroit Lions are the best team in football. That Super Bowl's 154 days away. There's a whole lot that has to happen before you even get close to touching that game in New Orleans. They've got to handle their business right here, right now. Come on, 5A! It's time to f***ing eat! All that f***ing work is for this s*** right here! And for this s*** right here, we're not the same! This different around here! Y'all need some inspiration? I'm gonna show y'all. Y'all need some f***ing standard? I'm gonna show y'all. One, two, three! One, two, three! We're in! Beat him up! Beat him up! Beat him up! Beat him up! I was here doubling the projects, mad at myself for putting it to the side. Mama had to cater for the coop that we rode after school on the way to 

In [12]:
# Replace with your API key
aai.settings.api_key = assemblyai_api_key

# URL of the file to transcribe
FILE_URL = "https://assembly.ai/wildfires.mp3"

# You can also transcribe a local file by passing in a file path
# FILE_URL = './path/to/file.mp3'

transcriber = aai.Transcriber()
transcript = transcriber.transcribe(audio_file)

print(transcript.text)

Downtown Detroit ready to open the 2024 season with higher hopes and more expectations that we've seen with this franchise in decades. Matter of fact, around the country, people are saying this is the team to beat. The Detroit Lions are the best team in football. That Super bowl is 154 days away. There's a whole lot that has to happen before you even get close to touching that game in New Orleans. They gotta handle their business right here, right now. All that work is for this right here. It's for this right here. We not the same get different around here. Y'all need some inspiration I'mma show y'all Y'all need some standards I'mma show y'all one on three One, two, three Beat em up, beat em up beat em up, beat em up I was seeing double in the projects mad at myself learn to put it to the side Mama had to cater for the coop that we rode at the school on the way to Popeyes and niggas want to play both sides just a red dot don't get on the wrong red eye a headshot and can't woo them guys

In [ ]:
def identify_song_from_lyrics(lyrics):
    # Initialize the client
    client = OpenAI(api_key=openai_api_key)
    # Create the prompt
    prompt = f"Identify the song from the following lyrics:\n\n{lyrics}\n\nSong name:"
    # Make the API call using the new format
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    # Extract and return the response
    return response.choices[0].message.content.strip()

lyrics = transcript.text
song_name = identify_song_from_lyrics(lyrics)
print("Identified Song Name:", song_name)

Identified Song Name: The lyrics provided are from the song "Family Ties" by Baby Keem and Kendrick Lamar.


In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True  # Enable long-form generation with timestamps
)

dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]

In [ ]:
# result = pipe(sample)  
# audio_file = '../data/track_audio.mp3'
result = pipe(audio_file)

print(result["text"])


In [ ]:
# client = OpenAI(
#   api_key=openai_api_key,
# )

# completion = client.chat.completions.create(
#   model="gpt-4o-mini",
#   store=True,
#   messages=[
#     {"role": "user", "content": "write a haiku about ai"}
#   ]
# )

# print(completion.choices[0].message);


In [ ]:
# !pip freeze | sed 's/ @ file:.*//' > ../requirements.txt